# The effect of winning an NBA game at the buzzer

On Basketball Reference, you can find a [complete list](https://www.basketball-reference.com/friv/buzzer-beaters.html) of all game-winning buzzer beaters in NBA history. We've seen three so far this season, including a Kobe-esque fadeaway buzzer beater by Giannis Antetekoumpo:

<img src="img/kobe.jpg" align="left" width = 480/><img src="img/giannis.jpeg" align="left" width = 530/>


With these games in mind, I wanted to look at whether there is a *let-down effect* for the team that just won a game at the buzzer. That is, whether the emotional high that comes from such a thrilling victory leads to a slump in performance the following game. Here is my basic statistical setup:
<br>

**Null hypothesis**: Winning a game at the buzzer has no effect on the outcome of the team's next game.
<br>
**Alternative Hypothesis**: A team's chances of winning it's next game goes down after a game-winning buzzer beater.

In [20]:
import nba_api.stats.endpoints as e
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.options.display.width = 0
pd.set_option('display.float_format', '{:.3f}'.format)
pd.options.display.max_colwidth = 200

Reading in two dataframes: R has Game Logs data, df has buzzer beater data

In [2]:
R = pd.read_csv("TeamGameLogs_raw.csv")

Renaming columns and fixing dates

In [3]:
def fix_date_col(s):
    if s.endswith('T00:00:00'):
        return s[:-9]
    else:
        return s

R['date'] = R['date'].apply(fix_date_col)
R["date"] = pd.to_datetime(R['date'])
R = R.rename(columns = {"TEAM_ABBREVIATION":"team"})
R['opp'] = R['MATCHUP'].apply(lambda x: x.split(' ')[-1])



Pulling Buzzer Beater data

In [4]:
df = pd.read_csv("bref_cleaned.tsv", sep="\t")


def fix_date(s):
    if s[-2:] ==' p':
        return s[:-2]
    else:
        return s
df['date'] = df['Game'].apply(fix_date)
df['date'] = pd.to_datetime(df['date'])
df = df.rename(columns = {"Team":"team","Season":"season","Opp":"opp"})
df['buzzer_beater'] = True

Replacing team abbreviations that are in the basketball reference that are not in the NBA game logs data

In [5]:
bref_to_nba =  {'BLB':'BAL',
                'BRK':'BKN',
                'CHO':'CHA',
                'KCO':'KCK',
                'MLH':'MIH',
                'PHO':'PHX',
                'STB':'BOM',
                'WSB':'WAS',
                'WSC':'WAS'}


df['team'] = df['team'].replace(bref_to_nba)
df['opp'] = df['opp'].replace(bref_to_nba)

Replacing team abbreviations that are found in the Game Logs dataset that dont match the basketball reference abbreviation

In [7]:

nba_to_bref = { 'BLT':"BAL",
 'GOS':"GSW",
 'PHL':"PHI",
 'SAN':"SAS",
 'UTH':"UTA"}

R['team'] = R['team'].replace(nba_to_bref)
R['opp'] = R['opp'].replace(nba_to_bref)

Merge the two frames

In [9]:
R_to_merge = R.loc[:,['team','opp','MATCHUP','WL','MIN','FGM','FGA','FG_PCT','PF','PTS','PLUS_MINUS','season','date','stype']].copy()
df_to_merge = df.loc[:,['date','Player','team','Margin','Type','Assisted','Distance','buzzer_beater']].copy()
df_to_merge = df_to_merge.sort_values(['date','team'])
Merged = pd.merge(R_to_merge,df_to_merge, on = ['date','team'],how = 'left')
Merged['buzzer_beater'] = Merged['buzzer_beater'].fillna(False)

Rename Merged to df

In [10]:
df = Merged
df = df.sort_values(['date','team'])

Make a boolean column that is True if the previous game of the team was a buzzer beater

In [11]:
df['buzzer_beater_prev'] = df.groupby(['season','team'])['buzzer_beater'].shift()
df['buzzer_beater_prev'] = df['buzzer_beater_prev'].fillna(False)

Make a days rest column that gives the number of days passed since the pervious game

In [12]:
df['rest'] = df.groupby(['season','team'])['date'].diff().dt.days

Make a 0-1 column that is 1 if win, 0 if False

In [13]:
df['win'] = (df['PLUS_MINUS']>0).astype(int)

Make win running average win percentage column

In [14]:
df['win_pct'] = df.groupby(['season','team'])['win'].transform(lambda x: x.expanding().mean().shift())

Get the opponent's win percentage running average heading into the game

In [15]:
df['is_home'] = df['MATCHUP'].str.contains("vs.")
H = df[df['is_home']]
A = df[~df['is_home'].astype(bool)]
P = pd.merge(H,A.loc[:,['date','opp','win_pct']],how = 'left',left_on = ['date','team'],right_on = ['date','opp'],suffixes = ['','_opp'])
Q = pd.merge(A,H.loc[:,['date','opp','win_pct']],how = 'left',left_on = ['date','team'],right_on = ['date','opp'],suffixes = ['','_opp'])
PP = pd.concat([P,Q]).sort_values('date')
PP = PP.drop(columns = 'opp_opp')
PP = PP.reset_index(drop=True)
df = PP

## Analyzing the data

In [16]:
A = df['buzzer_beater_prev']
B = df['buzzer_beater']
T = df.loc[(A|B),['date','MATCHUP','rest','Player','Margin','Type','PLUS_MINUS','win']].set_index('date')
TT = df.loc[(A),['date','MATCHUP','rest','Player','Margin','Type','PLUS_MINUS','win']].set_index('date')

In [17]:
H = T.tail(15).loc[:,['MATCHUP','Player','Margin','Type','PLUS_MINUS']]
H.iloc[[0,1,2,3,4,5,6,7,8,9,11,10,12,13,14]]

,MATCHUP,Player,Margin,Type,PLUS_MINUS
date,,,,,
2025-03-27,CHI vs. LAL,Josh Giddey,-1,3-pt FG,2.000
2025-03-29,CHI vs. DAL,NaN,NaN,NaN,-1.000
2025-04-03,MEM @ MIA,Ja Morant (2),tied,2-pt FG,2.000
2025-04-05,MEM @ DET,NaN,NaN,NaN,6.000
2025-04-09,SAS @ GSW,Harrison Barnes (4),tied,3-pt FG,3.000
2025-04-11,SAS @ PHX,NaN,NaN,NaN,-19.000
2025-04-13,WAS @ MIA,Bub Carrington,-1,2-pt FG,1.000
2025-04-26,DEN @ LAC,Aaron Gordon,tied,2-pt FG,2.000
2025-04-29,DEN vs. LAC,NaN,NaN,NaN,16.000


Make a new dataframe that contains z scores for win loss records of teams following a buzzer beater

In [21]:
seasons =  [str(i)+'-'+str(i+1)[2:] for i in range(1946,2026)]
data = []
A = df['buzzer_beater_prev']
B = df['stype'] == 'Regular Season'
for i,season in enumerate(seasons[:-3]):
    #print("For seasons" ,season,"and on:")
    C = df['season'].isin(seasons[i:])
    FILTER = A&B&C
    SLICE = df.loc[FILTER]
    W = SLICE['win'].sum()
    n = SLICE.shape[0]
    L = n-W
    z = (W/n - .5) / np.sqrt(.5 * (1 - .5) / n)
    data.append([season,W,L,n,z,'regular season'])

B = df['stype'] == 'Playoffs'
for i,season in enumerate(seasons[:-3]):
    #print("For seasons" ,season,"and on:")
    C = df['season'].isin(seasons[i:])
    FILTER = A&B&C
    SLICE = df.loc[A&B&C]
    W = SLICE['win'].sum()
    n = SLICE.shape[0]
    L = n-W
    z = (W/n - .5) / np.sqrt(.5 * (1 - .5) / n)
    data.append([season,W,L,n,z,'playoffs'])

cols = ['season-to-present','wins','losses','num_games','zscore','stype']
DF = pd.DataFrame(data,columns=cols)

In [26]:
DF[DF['season-to-present']=='1946-47']

,season-to-present,wins,losses,num_games,zscore,stype
0,1946-47,399,392,791,0.249,regular season
77,1946-47,22,35,57,-1.722,playoffs


In [27]:
DF[DF['season-to-present']=='2000-01']

,season-to-present,wins,losses,num_games,zscore,stype
54,2000-01,175,175,350,0.000,regular season
131,2000-01,14,17,31,-0.539,playoffs


In [30]:
DF[DF['season-to-present']=='2014-15']

,season-to-present,wins,losses,num_games,zscore,stype
68,2014-15,65,78,143,-1.087,regular season
145,2014-15,8,9,17,-0.243,playoffs


In [31]:
DF[DF['season-to-present']=='2022-23']

,season-to-present,wins,losses,num_games,zscore,stype
76,2022-23,13,30,43,-2.592,regular season
153,2022-23,2,1,3,0.577,playoffs


In [33]:
DF[DF['season-to-present'].isin(['1946-47','2000-01','2022-23'])]

,season-to-present,wins,losses,num_games,zscore,stype
0,1946-47,399,392,791,0.249,regular season
54,2000-01,175,175,350,0.000,regular season
76,2022-23,13,30,43,-2.592,regular season
77,1946-47,22,35,57,-1.722,playoffs
131,2000-01,14,17,31,-0.539,playoffs
153,2022-23,2,1,3,0.577,playoffs


In [53]:
seasons =  [str(i)+'-'+str(i+1)[2:] for i in range(1946,2026)]
data = []
A = df['buzzer_beater_prev']
B = df['stype'] == 'Regular Season'
zipped = list(zip(range(len(seasons)),seasons))
for (i,season) in zipped[:-4:5]:
    #print("For seasons" ,season,"and on:")
    C = df['season'].isin(seasons[i:i+5])
    FILTER = A&B&C
    SLICE = df.loc[FILTER]
    W = SLICE['win'].sum()
    n = SLICE.shape[0]
    L = n-W
    z = (W/n - .5) / np.sqrt(.5 * (1 - .5) / n)
    data.append([season+'-to-'+zipped[i+4][1],W,L,n,z,'regular season'])

#B = df['stype'] == 'Playoffs'
#for (i,season) in zipped[:-9:10]:
#    #print("For seasons" ,season,"and on:")
#    C = df['season'].isin(seasons[i:i+10])
#    FILTER = A&B&C
#    SLICE = df.loc[A&B&C]
#    W = SLICE['win'].sum()
#    n = SLICE.shape[0]
#    L = n-W
#    z = (W/n - .5) / np.sqrt(.5 * (1 - .5) / n)
#    data.append([season+'-through-'+zipped[i+9][1],W,L,n,z,'playoffs'])

cols = ['season-group','wins','losses','num_games','zscore','stype']
DF = pd.DataFrame(data,columns=cols)

In [57]:
DF.set_index('season-group').loc[:,['wins','losses','zscore']].tail(5)

,wins,losses,zscore
season-group,,,
2001-02-to-2005-06,45,29,1.860
2006-07-to-2010-11,43,43,0.000
2011-12-to-2015-16,34,30,0.500
2016-17-to-2020-21,29,31,-0.258
2021-22-to-2025-26,21,35,-1.871


In [61]:
from scipy.stats import norm

DF['H0_likelihood'] = DF['zscore'].apply(norm.cdf)

In [62]:
DF.set_index('season-group').loc[:,['wins','losses','zscore','H0_likelihood']].tail(5)

,wins,losses,zscore,H0_likelihood
season-group,,,,
2001-02-to-2005-06,45,29,1.860,0.969
2006-07-to-2010-11,43,43,0.000,0.500
2011-12-to-2015-16,34,30,0.500,0.691
2016-17-to-2020-21,29,31,-0.258,0.398
2021-22-to-2025-26,21,35,-1.871,0.031
